# Aerospike


Aerospike é um banco de dados multi-modelo. Os experimentos a seguir mostram um comandos básicos de criação de um grafo simples, e como interagir usando esse sistema. 



### Deploy

```
docker run -d --name aerospike \
  -p 3000-3002:3000-3002 \
  aerospike/aerospike-server
  
  
# Ver informações do servidor
docker exec aerospike asinfo -v status  
```

###  Limitações da Community Edition

Segundo a documentação oficial, a **Community Edition** tem as seguintes restrições:


| Aspecto | Community Edition | Enterprise Edition |
| :-- | :-- | :-- |
| **Cluster Size** | Máximo 8 nós | Até 256 nós |
| **Data Limit** | Até 2.5 TB | Ilimitado |
| **Namespaces** | Máximo 2 | Até 32 |

## Abordagem 1 (multi-modelo)

Doc: https://aerospike-python-client.readthedocs.io/en/latest/

In [1]:
! pip install aerospike

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 5.9 MB/s eta 0:00:00 MB/s eta 0:00:01:03

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import aerospike

config = {
    'hosts': [('localhost', 3000)]
}

client = aerospike.client(config).connect()

In [3]:
def criar_vertice(vertex_id, properties):
    """Criar um vértice no grafo"""
    key = ('test', 'vertices', vertex_id)
    record = {
        'type': 'vertex',
        'properties': properties,
        'edges_out': [],  # Lista de arestas saindo
        'edges_in': []    # Lista de arestas entrando
    }
    client.put(key, record)
    return vertex_id

def criar_aresta(edge_id, from_vertex, to_vertex, relationship_type, properties=None):
    """Criar uma aresta entre dois vértices"""
    # Criar registro da aresta
    edge_key = ('test', 'edges', edge_id)
    edge_record = {
        'type': 'edge',
        'from': from_vertex,
        'to': to_vertex,
        'relationship': relationship_type,
        'properties': properties or {}
    }
    client.put(edge_key, edge_record)
    
    # Atualizar listas de adjacência nos vértices
    # Vértice origem
    from_key = ('test', 'vertices', from_vertex)
    try:
        (key, metadata, from_record) = client.get(from_key)
        from_record['edges_out'].append({
            'edge_id': edge_id,
            'to': to_vertex,
            'type': relationship_type
        })
        client.put(from_key, from_record)
    except aerospike.exception.RecordNotFound:
        print(f"Vértice {from_vertex} não encontrado")
    
    # Vértice destino
    to_key = ('test', 'vertices', to_vertex)
    try:
        (key, metadata, to_record) = client.get(to_key)
        to_record['edges_in'].append({
            'edge_id': edge_id,
            'from': from_vertex,
            'type': relationship_type
        })
        client.put(to_key, to_record)
    except aerospike.exception.RecordNotFound:
        print(f"Vértice {to_vertex} não encontrado")


In [4]:

# Criar vértices (pessoas)
criar_vertice('alice', {'name': 'Alice', 'age': 30, 'city': 'São Paulo'})
criar_vertice('bob', {'name': 'Bob', 'age': 25, 'city': 'Rio de Janeiro'})
criar_vertice('charlie', {'name': 'Charlie', 'age': 35, 'city': 'São Paulo'})

# Criar arestas (relacionamentos)
criar_aresta('edge1', 'alice', 'bob', 'FRIENDS', {'since': '2020'})
criar_aresta('edge2', 'bob', 'charlie', 'FRIENDS', {'since': '2021'})
criar_aresta('edge3', 'alice', 'charlie', 'COLLEAGUES', {'department': 'Engineering'})

print("Grafo social criado com sucesso!")


Grafo social criado com sucesso!


In [5]:

def consultar_amigos(vertex_id):
    """Consultar amigos de uma pessoa"""
    key = ('test', 'vertices', vertex_id)
    try:
        (key, metadata, record) = client.get(key)
        amigos = []
        for edge in record['edges_out']:
            if edge['type'] == 'FRIENDS':
                amigos.append(edge['to'])
        return amigos
    except aerospike.exception.RecordNotFound:
        return []


# Consultar amigos da Alice
amigos_alice = consultar_amigos('alice')
print(f"Amigos da Alice: {amigos_alice}")

Amigos da Alice: ['bob']


In [10]:
def encontrar_triangulo_aerospike(client):
    """
    Procura triângulos no grafo armazenado no Aerospike Database
    """
    # Obter todas as pessoas
    pessoas = []
    
    # Scan para encontrar todos os vértices do tipo pessoa
    def scan_callback(input_tuple):
        (key, metadata, record) = input_tuple
        if record.get('type') == 'vertex' and 'name' in record.get('properties', {}):
            pessoas.append({
                'id': key[2], 
                'name': record['properties']['name'],
                'amigos': [edge['to'] for edge in record.get('edges_out', []) 
                          if edge.get('type') == 'FRIENDS']
            })
    
    # Executar scan
    scan = client.scan('test', 'vertices')
    scan.foreach(scan_callback)
    
    # Procurar triângulos
    triangulos = []
    
    for i, pessoa_a in enumerate(pessoas):
        for j, pessoa_b in enumerate(pessoas[i+1:], i+1):
            for k, pessoa_c in enumerate(pessoas[j+1:], j+1):
                # Verificar se A conhece B, B conhece C, e C conhece A
                a_conhece_b = pessoa_b['id'] in pessoa_a['amigos']
                b_conhece_c = pessoa_c['id'] in pessoa_b['amigos']
                c_conhece_a = pessoa_a['id'] in pessoa_c['amigos']
                
                if a_conhece_b and b_conhece_c and c_conhece_a:
                    triangulos.append({
                        'pessoa1': pessoa_a['name'],
                        'pessoa2': pessoa_b['name'], 
                        'pessoa3': pessoa_c['name']
                    })
    
    return triangulos

# Conectar e executar
config = {'hosts': [('localhost', 3000)]}
client = aerospike.client(config).connect()

triangulos_encontrados = encontrar_triangulo_aerospike(client)

if triangulos_encontrados:
    print(f"Encontrados {len(triangulos_encontrados)} triângulos de amizade:")
    for trio in triangulos_encontrados:
        print(f"- {trio['pessoa1']}, {trio['pessoa2']}, {trio['pessoa3']}")
else:
    print("Nenhum triângulo de 3 pessoas que se conhecem mutuamente foi encontrado.")


Nenhum triângulo de 3 pessoas que se conhecem mutuamente foi encontrado.


In [6]:
# Fechar conexão
client.close()

## Abordagem 2 (Aerospike graph)

A versão do Aerospike graph (que teria acesso ao Apache Tinkerpop/Gremlin) requer uma licença! 

*An Aerospike feature key file which has the graph-service key enabled. Click here to get a free 60 day trial key, or contact your Aerospike account manager.* 

Fonte: https://aerospike.com/docs/graph/install/docker